In [1]:
%%bash

jps

11363 SecondaryNameNode
11539 Master
11044 NameNode
22586 Jps
11612 Worker
11725 LivyServer
11167 DataNode


In [2]:
%%bash

hdfs dfs -ls /consulta_parquet

Found 150 items
-rw-r--r--   1 saxsa supergroup          0 2019-08-05 23:38 /consulta_parquet/_SUCCESS
-rw-r--r--   1 saxsa supergroup        654 2019-08-05 23:37 /consulta_parquet/part-00000-fc69a552-8608-4a3a-bec3-caf99c434222-c000.snappy.parquet
-rw-r--r--   1 saxsa supergroup        645 2019-08-05 23:37 /consulta_parquet/part-00001-fc69a552-8608-4a3a-bec3-caf99c434222-c000.snappy.parquet
-rw-r--r--   1 saxsa supergroup        644 2019-08-05 23:37 /consulta_parquet/part-00002-fc69a552-8608-4a3a-bec3-caf99c434222-c000.snappy.parquet
-rw-r--r--   1 saxsa supergroup        644 2019-08-05 23:37 /consulta_parquet/part-00003-fc69a552-8608-4a3a-bec3-caf99c434222-c000.snappy.parquet
-rw-r--r--   1 saxsa supergroup        643 2019-08-05 23:37 /consulta_parquet/part-00004-fc69a552-8608-4a3a-bec3-caf99c434222-c000.snappy.parquet
-rw-r--r--   1 saxsa supergroup        643 2019-08-05 23:37 /consulta_parquet/part-00005-fc69a552-8608-4a3a-bec3-caf99c434222-c000.snappy.parquet
-rw-r--r--   1 saxsa 

In [3]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,None,pyspark3,idle,,,✔


SparkSession available as 'spark'.
<SparkContext master=spark://hostsaxsa:7077 appName=livy-session-7>

In [4]:
df_par = spark.read.parquet('/consulta_parquet')

In [5]:
df_par.sort('count', ascending=False).show()

+-----------+-----+
|Producto_ID|count|
+-----------+-----+
|       2233|94763|
|       1242|85109|
|       1250|77308|
|      35305|72972|
|       1240|69454|
|       1146|68716|
|       1109|65984|
|       1232|64003|
|       1125|62965|
|       1278|61439|
|      43069|61207|
|      43285|58953|
|       1284|56728|
|      30532|55688|
|      35651|52796|
|       1212|51713|
|       1238|50749|
|       1150|50480|
|       1309|48998|
|      37361|48087|
+-----------+-----+
only showing top 20 rows

In [6]:
df_mon = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("spark.mongodb.input.uri", "mongodb://127.0.0.1/bimbodb.producto?readPreference=primaryPreferred")\
    .option("spark.mongodb.output.uri", "mongodb://127.0.0.1/bimbodb.producto").load()

In [7]:
df_mon = df_mon.drop('_id')
df_mon.show()

+--------------------+-----------+
|      NombreProducto|Producto_ID|
+--------------------+-----------+
|   NO IDENTIFICADO 0|          0|
|Capuccino Moka 75...|          9|
|Bimbollos Ext sAj...|         41|
|Burritos Sincro 1...|         53|
|Div Tira Mini Dor...|         72|
|Pan Multigrano Li...|         73|
|Tostado Integral ...|         98|
|Pan Blanco 567g W...|         99|
|Super Pan Bco Ajo...|        100|
|Wonder 100pct med...|        106|
|Wonder 100pct gde...|        107|
|Baguette Precocid...|        108|
|Pan Multicereal 4...|        109|
|Tostado Integral ...|        112|
|Biscotel Receta O...|        122|
|Super Bollos 5in ...|        123|
|Bollos 8p 450g WO...|        125|
|Bollos BK 4in 36p...|        131|
|Bollos BK 5in 30p...|        132|
|Bollos BK 4in 30p...|        134|
+--------------------+-----------+
only showing top 20 rows

In [8]:
df_my = spark.read.format("jdbc").options(
    url="jdbc:mysql://localhost/bimbodb",
    driver="com.mysql.jdbc.Driver",
    dbtable="test",
    user="saxsauser",
    password="ggsaXsa").load()

In [9]:
df_my.show()

+-------+------+----------+--------+--------+----------+-----------+
|     id|Semana|Agencia_ID|Canal_ID|Ruta_SAK|Cliente_ID|Producto_ID|
+-------+------+----------+--------+--------+----------+-----------+
|1232998|    10|      1420|       1|    1206|   1875998|      34213|
|6662861|    11|      4010|       1|    2106|   1661843|      30549|
|4007443|    10|      1375|       1|    2008|   1234559|      43307|
|4840434|    10|      1531|       1|    2111|    158133|      35303|
|5379458|    11|      1544|       1|    1231|   2309256|       2233|
|4230113|    11|      1629|       1|    1120|   4527410|      41938|
|4947304|    10|      1470|       1|    2012|   4435970|      36610|
|2430839|    10|      1122|       1|    1056|   2055889|       1109|
| 922781|    10|      2032|       1|    1160|   4200375|       1212|
| 874484|    11|      1216|       1|    2126|   4522344|      35305|
|4004454|    10|      1315|       1|    2857|   4573216|      37361|
|3222214|    10|      1955|       

In [10]:
from pyspark.sql.functions import col
x = df_par.filter((col("Producto_ID") == 1250) | (col("Producto_ID") == 1242) | (col("Producto_ID") == 2233))
y = df_mon.filter((col('Producto_ID') == 1250) | (col('Producto_ID') == 1242) | (col("Producto_ID") == 2233))
x.join(y, x.Producto_ID == y.Producto_ID, 'outer').show()

+-----------+-----+--------------------+-----------+
|Producto_ID|count|      NombreProducto|Producto_ID|
+-----------+-----+--------------------+-----------+
|       1242|85109|Donitas Espolvore...|       1242|
|       1250|77308|Donas Azucar 4p 1...|       1250|
|       2233|94763|Pan Blanco 640g B...|       2233|
+-----------+-----+--------------------+-----------+